In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [3]:
from mcr_py import run_mlc_with_bags, GraphCache

In [4]:
from copy import deepcopy
from typing import Any, Tuple

import pandas as pd
import geopandas as gpd

import mcr_py
from mcr_py import GraphCache
import pyrosm
from package import storage, strtime
from package.logger import Timed
from package.mcr.label import McRAPTORLabel, merge_intermediate_bags
from package.mcr.path import PathManager, PathType
from package.osm import osm, graph
from package.raptor.mcraptor_single import McRaptorSingle
from package.raptor.bag import Bag
from package.mcr.bag import (
    convert_mc_raptor_bags_to_intermediate_bags,
    convert_mlc_bags_to_intermediate_bags,
)



ACCURACY = 1
ACCURACY_MULTIPLIER = 10 ** (ACCURACY - 1)

AVG_WALKING_SPEED = 1.4  # m/s
AVG_BIKING_SPEED = 4.0  # m/s


In [5]:
def prefix_id(
    gdf: pd.DataFrame, prefix: str, column: str, save_old=False
) -> pd.DataFrame:
    if save_old:
        gdf[f"{column}_old"] = gdf[column]
    gdf[column] = prefix + gdf[column].astype(str)

    return gdf


def get_graph(
    osm_reader: pyrosm.OSM, stops_df: gpd.GeoDataFrame
) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    with Timed.info("Getting OSM graph"):
        nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)

    return nodes, edges


def mark_bicycles(nodes: pd.DataFrame) -> pd.DataFrame:
    nodes["has_bicycle"] = False
    nodes.loc[nodes.sample(100).index, "has_bicycle"] = True
    return nodes


def create_multi_modal_graph(
    nodes: pd.DataFrame, edges: pd.DataFrame
) -> dict[str, pd.DataFrame]:
    edges = add_reverse_edges(edges)

    walking_nodes = nodes.copy()
    bike_nodes = nodes.copy()
    walking_edges = edges.copy()
    bike_edges = edges.copy()

    walking_nodes = prefix_id(walking_nodes, "W", "id", save_old=True)
    bike_nodes = prefix_id(bike_nodes, "B", "id", save_old=True)

    walking_edges = prefix_id(walking_edges, "W", "u")
    walking_edges = prefix_id(walking_edges, "W", "v")
    bike_edges = prefix_id(bike_edges, "B", "u")
    bike_edges = prefix_id(bike_edges, "B", "v")

    transfer_edges = create_transfer_edges(nodes)

    walking_edges = add_travel_time(walking_edges, AVG_WALKING_SPEED)
    bike_edges = add_travel_time(bike_edges, AVG_BIKING_SPEED)

    bike_edges["travel_time_bike"] = bike_edges["travel_time"]

    edges = combine_edges(walking_edges, bike_edges, transfer_edges)
    nodes = pd.concat([walking_nodes, bike_nodes])
    return {
        "nodes": nodes,
        "edges": edges,
        "walking_nodes": walking_nodes,
        "walking_edges": walking_edges,
    }


def add_reverse_edges(edges: pd.DataFrame) -> pd.DataFrame:
    reverse_edges = edges.copy()
    reverse_edges = reverse_edges.rename(columns={"u": "v", "v": "u"})
    return pd.concat([edges, reverse_edges])



# create transfer edges from bike to walk at all nodes
def create_transfer_edges(nodes: pd.DataFrame):
    transfer_edges_values: pd.Series = nodes.apply(
        lambda x: ["B" + str(x.id), "W" + str(x.id), 0], axis=1
    )  # type: ignore
    transfer_edges = pd.DataFrame(
        transfer_edges_values.tolist(), columns=["u", "v", "length"]
    )

    return transfer_edges


def add_travel_time(edges: pd.DataFrame, speed: float) -> pd.DataFrame:
    edges["travel_time"] = edges.length / speed

    return edges


def combine_edges(
    walking_edges: pd.DataFrame,
    bike_edges: pd.DataFrame,
    transfer_edges: pd.DataFrame,
) -> pd.DataFrame:
    edges = pd.concat([walking_edges, bike_edges, transfer_edges], ignore_index=True)

    # fill travel_time for transfer edges and
    # travel_time_bike for walking and transfer edges
    edges = edges.fillna(0)

    return edges


def add_multi_modal_weights(edges: pd.DataFrame) -> pd.DataFrame:
    edges["weights"] = (
        "("
        + (edges["travel_time"].round(ACCURACY) * ACCURACY_MULTIPLIER)
        .astype(int)
        .astype(str)
        + ",0)"
    )
    edges["hidden_weights"] = (
        "("
        + (edges["travel_time_bike"].round(ACCURACY) * ACCURACY_MULTIPLIER)
        .astype(int)
        .astype(str)
        + ")"
    )

    return edges


def add_single_modal_weights(edges: pd.DataFrame) -> pd.DataFrame:
    edges["weights"] = (
        "("
        + (edges["travel_time"].round(ACCURACY) * ACCURACY_MULTIPLIER)
        .astype(int)
        .astype(str)
        + ",0)"
    )
    return edges


def reset_node_ids(
    nodes: pd.DataFrame, edges: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame, dict[Any, int]]:
    node_map = {}
    for i, node_id in enumerate(nodes.id.unique()):
        node_map[node_id] = i

    nodes["old_id"] = nodes["id"]
    nodes["id"] = nodes["id"].map(node_map)
    edges["u"] = edges["u"].map(node_map)
    edges["v"] = edges["v"].map(node_map)

    total_na = edges.isna().sum().sum() + nodes.isna().sum().sum()
    if total_na > 0:
        raise ValueError(f"Found {total_na} NaNs in graph")

    return nodes, edges, node_map


def get_reverse_map(d: dict[Any, Any]) -> dict[Any, Any]:
    return {v: k for k, v in d.items()}



In [6]:
stops_path = "../data/cleaned/stops.csv"
city_id = "Koeln"
osm_path = ""
structs="../data/structs.pkl"

# structs_dict = storage.read_any_dict(structs)
# with Timed.info("Reading stops"):
#     stops_df = storage.read_gdf(stops_path)

# with Timed.info("Preparing graphs"):
#     osm_reader = osm.get_osm_reader_for_city_id_or_osm_path(city_id, osm_path)
#     nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)
#     nxgraph = graph.create_nx_graph(osm_reader, nodes, edges)

#     nodes: pd.DataFrame = nodes[["id"]]  # type: ignore
#     edges: pd.DataFrame = edges[["u", "v", "length"]]  # type: ignore

#     stops_df = graph.add_nearest_node_to_stops(stops_df, nxgraph)

#     stops_df["stop_id"] = stops_df["stop_id"].astype(int)
#     stop_node_map = stops_df.set_index("stop_id")["nearest_node"].to_dict()
#     node_stop_map = {v: k for k, v in stop_node_map.items()}

#     nodes = mark_bicycles(nodes)

#     graph_components = create_multi_modal_graph(nodes, edges)
#     nodes, edges, walking_nodes, walking_edges = (
#         graph_components["nodes"],
#         graph_components["edges"],
#         graph_components["walking_nodes"],
#         graph_components["walking_edges"],
#     )

#     nodes, edges, node_map = reset_node_ids(nodes, edges)
#     walking_nodes, walking_edges, walking_node_map = reset_node_ids(
#         walking_nodes, walking_edges
#     )
#     reverse_node_map = get_reverse_map(node_map)
#     reverse_walking_node_map = get_reverse_map(walking_node_map)

#     edges = add_multi_modal_weights(edges)
#     walking_edges = add_single_modal_weights(walking_edges)

#     raw_edges = edges[["u", "v", "weights", "hidden_weights"]].to_dict("records")
#     raw_walking_edges = walking_edges[["u", "v", "weights"]].to_dict("records")

# bicycle_transfer_nodes_walking_node_ids = walking_nodes[
#     walking_nodes["has_bicycle"]
# ].id.values

# with Timed.info("Creating graph cache"):
#     gc = GraphCache()
#     gc.set_graph(raw_edges)
#     walking_gc = GraphCache()
#     walking_gc.set_graph(raw_walking_edges)

# with Timed.info("Running Dijkstra step"):
#     start_node_id = 295101994
#     walking_result_bags = mcr_py.run_mlc_with_node_and_time(
#         walking_gc,
#         walking_node_map[f"W{start_node_id}"],
#         strtime.str_time_to_seconds("08:00:00"),
#     )

# path_manager = PathManager()
# walking_result_bags = convert_mlc_bags_to_intermediate_bags(walking_result_bags)
# path_manager.extract_all_paths_from_bags(walking_result_bags, PathType.WALKING)

# # translates a node id from the walking graph to the corresponding bicycle
# # node id from the multi-modal graph
# def translate_walking_node_id_to_bicycle_node_id(
#     walking_node_id: int,
# ) -> int:
#     original_walking_node = reverse_walking_node_map[walking_node_id]
#     original_bicycle_node = original_walking_node.replace("W", "B")
#     bicycle_node_id = node_map[original_bicycle_node]
#     return bicycle_node_id

# # filter bags at bicycle nodes
# bicycle_bags = {
#     node_id: bag
#     for node_id, bag in walking_result_bags.items()
#     if node_id in bicycle_transfer_nodes_walking_node_ids
# }
# # translate node ids
# bicycle_bags = {
#     translate_walking_node_id_to_bicycle_node_id(node_id): [
#         label.to_mlc_label(translate_walking_node_id_to_bicycle_node_id(node_id))
#         for label in labels
#     ]
#     for node_id, labels in bicycle_bags.items()
# }

# # validation
# for node_id in bicycle_bags.keys():
#     gc.validate_node_id(node_id)

# bicycle_result_bags = mcr_py.run_mlc_with_bags(gc, bicycle_bags, update_label_func="next_bike_tariff")  # type: ignore
# bicycle_result_bags = convert_mlc_bags_to_intermediate_bags(bicycle_result_bags)
# path_manager.extract_all_paths_from_bags(
#     bicycle_result_bags, PathType.CYCLING_WALKING, path_index_offset=1
# )

# # --- McRAPTOR step
# def translate_walking_node_id_to_stop_id(walking_node_id: int) -> str | None:
#     original_walking_node = reverse_walking_node_map[walking_node_id]
#     original_bicycle_node = int(original_walking_node[1:])
#     if original_bicycle_node in node_stop_map:
#         stop_id = str(node_stop_map[original_bicycle_node])
#         return stop_id
#     return None

# mc_raptor_bags = {
#     node_id: bag
#     for node_id, bag in walking_result_bags.items()
#     if translate_walking_node_id_to_stop_id(node_id) is not None
# }
# mc_raptor_bags = {
#     translate_walking_node_id_to_stop_id(node_id): Bag.from_labels(
#         [
#             label.to_mc_raptor_label(
#                 translate_walking_node_id_to_stop_id(node_id), null_cost=True  # type: ignore
#             )
#             for label in labels
#         ]
#     )
#     for node_id, labels in mc_raptor_bags.items()
# }

# mc_raptor = McRaptorSingle(
#     structs_dict,
#     60,
#     McRAPTORLabel,
# )

# mc_raptor_result_bags = mc_raptor.run(mc_raptor_bags)  # type: ignore

# def translate_stop_id_to_node_id(stop_id: int) -> int:
#     node_id = stop_node_map[stop_id]
#     return node_map[f"W{node_id}"]

# mc_raptor_result_bags = {
#     translate_stop_id_to_node_id(int(stop_id)): bag
#     for stop_id, bag in mc_raptor_result_bags.items()
# }
# mc_raptor_result_bags = convert_mc_raptor_bags_to_intermediate_bags(
#     mc_raptor_result_bags,
#     min_path_length=2,
# )
# path_manager.extract_all_paths_from_bags(
#     mc_raptor_result_bags, PathType.PUBLIC_TRANSPORT, path_index_offset=1
# )

# combined_bags = deepcopy(
#     bicycle_result_bags
# )  # remove deepcopy, if you are sure, that bicycle_result_bags is not used anymore
# for node_id, bag in mc_raptor_result_bags.items():
#     merged_bag = (
#         merge_intermediate_bags(combined_bags[node_id], bag)
#         if node_id in combined_bags
#         else bag
#     )
#     combined_bags[node_id] = merged_bag
# for bag in combined_bags.values():
#     # nullify hidden_values
#     for label in bag:
#         label.hidden_values = []

# walking_result_bags_2 = mcr_py.run_mlc_with_bags(
#     walking_gc,
#     combined_bags,
# )
# walking_result_bags_2 = convert_mlc_bags_to_intermediate_bags(walking_result_bags_2)
# path_manager.extract_all_paths_from_bags(
#     walking_result_bags_2, PathType.WALKING, path_index_offset=2
# )

# storage.write_any_dict(
#     {
#         "walking_result_bags": walking_result_bags,
#         "walking_result_bags_2": walking_result_bags_2,
#         "path_manager": path_manager,
#         "bicycle_result_bags": bicycle_result_bags,
#         "mc_raptor_result_bags": mc_raptor_result_bags,
#         "mc_raptor_bags": mc_raptor_bags,
#         "combined_bags": combined_bags,
#         "node_map": node_map,
#         "walking_node_map": walking_node_map,
#         "stops_df": stops_df,
#     },
#     "/home/moritz/dev/uni/mcr-py/data/bags.pkl",  # type: ignore
# )


In [34]:
path_source = "/home/moritz/dev/uni/mcr-py/data/mcr5/Koeln/891fa1989cbffff.pkl"

data = storage.read_any_dict(path_source)
path_manager = data["path_manager"]
node_map = data["multi_modal_node_to_resetted_map"]
walking_node_map = data["walking_node_to_resetted_map"]
reverse_node_map = {v: k for k, v in node_map.items()}
reverse_walking_node_map = {v: k for k, v in walking_node_map.items()}
stops_df = data["stops_df"]
bags_i = data["bags_i"]


In [35]:
import os
import folium
from package.mcr.path import PathType

In [36]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, other_stops_df)

[10:06:55] INFO     Reading stops                                                                   ]8;id=377155;file:///tmp/ipykernel_15826/1792129641.py\1792129641.py]8;;\:]8;id=952081;file:///tmp/ipykernel_15826/1792129641.py#5\5]8;;\

           INFO     Reading stops done (0.05 seconds)                                               ]8;id=303720;file:///tmp/ipykernel_15826/1792129641.py\1792129641.py]8;;\:]8;id=648642;file:///tmp/ipykernel_15826/1792129641.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=5000;file:///tmp/ipykernel_15826/1792129641.py\1792129641.py]8;;\:]8;id=740238;file:///tmp/ipykernel_15826/1792129641.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=211795;file:///tmp/ipykernel_15826/1792129641.py\1792129641.py]8;;\:]8;id=396800;file:///tmp/ipykernel_15826/1792129641.py#16\16]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=566012;file:///tmp/ipykernel_15826/1792129641.py\1792129641.py]8;;\:]8;id=202674;file:///tmp/ipykernel_15826/1792129641.py#16\16]8;;\

[10:06:57] INFO     Getting OSM graph done (1.25 seconds)                                          ]8;id=274498;file:///tmp/ipykernel_15826/1792129641.py\1792129641.py]8;;\:]8;id=463214;file:///tmp/ipykernel_15826/1792129641.py#16\16]8;;\

In [37]:
labels = pd.DataFrame(
    [
        (label.node_id, label.values[0], label.values[1], n_transfers)
        for n_transfers, bags in bags_i.items()
        for bag in bags.values()
        for label in bag
    ],
    columns=["osm_node_id", "time", "cost", "n_transfers"],
)

labels["human_readable_time"] = labels["time"].apply(strtime.seconds_to_str_time)
labels


,osm_node_id,time,cost,n_transfers,human_readable_time
0,770387691,35513,0,0,09:51:53
1,10009928222,37601,0,0,10:26:41
2,258338924,30601,0,0,08:30:01
3,3335003564,37756,0,0,10:29:16
4,311638458,34471,0,0,09:34:31
...,...,...,...,...,...
552861,4395758147,31093,0,2,08:38:13
552862,10957722999,31582,0,2,08:46:22
552863,1588203481,30902,0,2,08:35:02
552864,5676336231,31373,1,2,08:42:53


In [38]:
labels[["time", "cost", "n_transfers"]].describe()

,time,cost,n_transfers
count,552866.000000,552866.000000,552866.000000
mean,33157.364459,0.337009,1.149620
std,2194.232911,0.490616,0.758278
min,28800.000000,0.000000,0.000000
25%,31669.000000,0.000000,1.000000
50%,32366.000000,0.000000,1.000000
75%,34205.000000,1.000000,2.000000
max,40998.000000,2.000000,2.000000


In [39]:
max_time = strtime.str_time_to_seconds("09:00:00")
max_cost = 10
max_transfers = 10

valid_labels = labels[
	(labels["time"] <= max_time)
	& (labels["cost"] <= max_cost)
	& (labels["n_transfers"] <= max_transfers)
]

In [40]:
selection = valid_labels.sample(min(1000, len(valid_labels)))
# selection = valid_labels

max_cost = selection.cost.max()

from typing import Dict
from colorsys import hls_to_rgb

def hsl_to_hex(h: float, l: float, s: float) -> str:
    r, g, b = [int(x * 255.0) for x in hls_to_rgb(h, l, s)]
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def generate_colors_hsl(max_cost: float) -> Dict[int, str]:
    if max_cost == 0:
        return {0: "#ffcccc"}
    return {
        cost: hsl_to_hex(h=0, l=0.5 + 0.4 * (max_cost - cost) / max_cost, s=0.8)
        for cost in range(int(max_cost) + 1)
    }

colors = generate_colors_hsl(max_cost)

In [41]:
start_node_id = labels[labels.time == labels.time.min()].iloc[0].osm_node_id


In [42]:
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index

start_node = nodes_by_id.loc[start_node_id]

m = folium.Map(location=[start_node.lat, start_node.lon], zoom_start=13)

for label in selection.itertuples():
	node = nodes_by_id.loc[label.osm_node_id]
	folium.CircleMarker(
		location=[node.lat, node.lon],
		radius=3,
		weight=1,
		fill=True,
		fill_color=colors[label.cost],
		fill_opacity=1,
		color="black",
		popup=f"{label.human_readable_time} ({label.cost})",
	).add_to(m)

folium.CircleMarker(
	location=[start_node.lat, start_node.lon],
	radius=5,
	weight=1,
	fill=True,
	fill_color="green",
	fill_opacity=1,
	color="black",
	popup=f"Start",
).add_to(m)

m

In [43]:
translator_map = {
    PathType.WALKING: reverse_walking_node_map,
    PathType.CYCLING_WALKING: reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}
no_prefix_reverse_walking_node_map = {
    k: int(v[1:]) for k, v in reverse_walking_node_map.items()
}
no_prefix_reverse_node_map = {k: int(v[1:]) for k, v in reverse_node_map.items()}
no_prefix_translator_map = {
    PathType.WALKING: no_prefix_reverse_walking_node_map,
    PathType.CYCLING_WALKING: no_prefix_reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}


In [44]:
from package.mcr.path import Path, GTFSPath

In [45]:
stops_df['stop_id'] = stops_df['stop_id'].astype(int)
stops_by_id = stops_df.set_index('stop_id')
stops_by_id.head(2)

,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,geometry,nearest_node,nearest_node_dist
stop_id,,,,,,,,,,,,,
1,,Köln Heumarkt,,50.935705,6.959995,,,0,,,POINT (6.96000 50.93570),673702632,16.337340
2,,Köln Neumarkt,,50.93577,6.947677,,,0,,,POINT (6.94768 50.93577),443861,17.695676


In [46]:
walking_result_bags = bags_i[2]

In [47]:
walking_result_bags_flat = []
for node_id, bag in walking_result_bags.items():
	for label in bag:
		walking_result_bags_flat.append((node_id, label))

In [48]:
path_objs_with_ids = pd.Series(
    list(
        map(
            lambda x: (
                x[0],
                path_manager.reconstruct_and_translate_path_for_label(
                    x[1], no_prefix_translator_map
                ),
            ),
            walking_result_bags_flat[:10000],
        )
    )
)
node_ids, path_objs = list(zip(*path_objs_with_ids))
path_objs[:3]

([<package.mcr.path.Path at 0x7fd523145090>,
  <package.mcr.path.Path at 0x7fd523145330>])

In [49]:
len(walking_result_bags_flat)

206493

In [50]:
node_ids = pd.Series(node_ids)
path_objs = pd.Series(list(path_objs))

In [51]:
df = pd.DataFrame(path_objs, columns=["paths"])
df["contains_gtfs"] = df["paths"].apply(
    lambda x: any([isinstance(path, GTFSPath) for path in x])
)
df["first_path_length"] = df["paths"].apply(lambda x: len(x[0].path))
df["third_path_length"] = df["paths"].apply(lambda x: len(x[2].path))
df["fourth_path_length"] = df["paths"].apply(lambda x: len(x[3].path) if isinstance(x[3], Path) else 0)
df["fifth_path_length"] = df["paths"].apply(lambda x: len(x[4].path))
# df["sixth_path_length"] = df["paths"].apply(lambda x: len(x[5].path) if isinstance(x[5], Path) else 0)
# df["seventh_path_length"] = df["paths"].apply(lambda x: len(x[6].path))
df["path_types"] = df["paths"].apply(
    lambda x: [path.path_type if isinstance(path, Path) else "GTFS" for path in x]
)
df["node_id"] = node_ids
df.head(3)


,paths,contains_gtfs,first_path_length,third_path_length,fourth_path_length,fifth_path_length,path_types,node_id
0,[<package.mcr.path.Path object at 0x7fd5231450...,False,53,0,102,0,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",650763088
1,[<package.mcr.path.Path object at 0x7fd5231452...,True,53,0,0,29,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",650763088
2,[<package.mcr.path.Path object at 0x7fd523144e...,False,53,0,425,0,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",3311728186


In [52]:
df[df.contains_gtfs].size / df.size

0.059

In [53]:
df[df.contains_gtfs].node_id.unique().size / df.node_id.unique().size

0.09808811305070657

In [54]:
df[df.contains_gtfs]

,paths,contains_gtfs,first_path_length,third_path_length,fourth_path_length,fifth_path_length,path_types,node_id
1,[<package.mcr.path.Path object at 0x7fd5231452...,True,53,0,0,29,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",650763088
3,[<package.mcr.path.Path object at 0x7fd5231453...,True,53,0,0,18,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",3311728186
5,[<package.mcr.path.Path object at 0x7fd5231457...,True,53,0,0,85,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",8137207912
39,[<package.mcr.path.Path object at 0x7fd51d5d98...,True,53,0,0,72,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",3265161786
67,[<package.mcr.path.Path object at 0x7fd514838d...,True,53,0,0,43,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",3823369996
...,...,...,...,...,...,...,...,...
9977,[<package.mcr.path.Path object at 0x7fd1cab0a0...,True,53,0,0,42,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",3823481058
9980,[<package.mcr.path.Path object at 0x7fd1cab0a6...,True,53,0,0,20,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",2392561706
9983,[<package.mcr.path.Path object at 0x7fd1cab0ab...,True,53,0,0,29,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",853159005
9984,[<package.mcr.path.Path object at 0x7fd1cab0ad...,True,53,0,0,3,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",4861510839


In [55]:
effective_by_public_transport = df[df.contains_gtfs].node_id.unique()

In [56]:
import random
i = random.sample(list(df.node_id.unique()), 1)[0]
# i = 113757
sel = [tu for tu in walking_result_bags_flat if tu[0] == i]
print(f"num GTFS paths = {df[df.node_id == i].contains_gtfs.sum()}")
print(i)

num GTFS paths = 0
1670869995


In [57]:
print("\n".join(map(str, walking_result_bags[i])))

IntermediateLabel(values=[30527, 0], hidden_values=[], path=[116664, 329956, 354490, 745158, 786850], node_id=1670869995)


In [58]:
from folium import plugins
from folium.plugins import HeatMap

In [59]:
toloop = sel

sample_label = sel[0][1]
sample_node_id = sample_label.node_id
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index
sample_node = nodes_by_id.loc[sample_node_id]

m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=13)

for end_node_id , label in toloop:

    end_node = nodes_by_id.loc[end_node_id]

    folium.CircleMarker(
        location=[end_node.lat, end_node.lon],
        popup=f"End: {end_node_id}",
        color="red",
        radius=3,
    ).add_to(m)

    paths = path_manager.reconstruct_and_translate_path_for_label(
        label, translator_map
    )
    for path in paths:
        if isinstance(path, Path):
            if path.path == []:
                continue
            cycling_path_nodes = [nodes_by_id.loc[int(node_id[1:])] for node_id in path.path if node_id[0] == "B"]
            walking_path_nodes = [nodes_by_id.loc[int(node_id[1:])] for node_id in path.path if node_id[0] == "W"]
            # print(f"cycling length: {len(cycling_path_nodes)}")
            # print(f"walking length: {len(walking_path_nodes)}\n")
            path_lat_lon = [(node.lat, node.lon) for node in cycling_path_nodes]
            if path_lat_lon != []:
                folium.PolyLine(path_lat_lon, color="blue", weight=2).add_to(m)
            path_lat_lon = [(node.lat, node.lon) for node in walking_path_nodes]
            if path_lat_lon != []:
                folium.PolyLine(path_lat_lon, color="red", weight=2).add_to(m)
        elif isinstance(path, GTFSPath):
            start_stop_id = path.start_stop_id
            end_stop_id = path.end_stop_id
            start_stop = stops_by_id.loc[start_stop_id]
            end_stop = stops_by_id.loc[end_stop_id]
            trip = path.trip_id

            path_lat_lon = [
                (float(start_stop.stop_lat), float(start_stop.stop_lon)),
                (float(end_stop.stop_lat), float(end_stop.stop_lon)),
            ]
            folium.PolyLine(
                path_lat_lon,
                color="green",
                weight=2,
                popup=f"Trip: {trip}",
            ).add_to(m)

            folium.CircleMarker(
                location=[float(start_stop.stop_lat), float(start_stop.stop_lon)],
                popup=f"Start: {start_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
            folium.CircleMarker(
                location=[float(end_stop.stop_lat), float(end_stop.stop_lon)],
                popup=f"End: {end_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
        else:
            raise Exception("Unknown path type")

m


In [60]:
m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=13)

heat_map_nodes = [nodes_by_id.loc[node_id] for node_id in effective_by_public_transport]
HeatMap(
    [(node.lat, node.lon) for node in heat_map_nodes],
).add_to(m)

for _, stop in stops_df.iterrows():
    node = nodes_by_id.loc[stop["nearest_node"]]

    folium.CircleMarker(
        location=[node.lat, node.lon],
        radius=3,
        popup=f"node_id: {node_id}",
        color="green",
    ).add_to(m)


m